In [1]:
import os

In [2]:
%pwd

'/home/karthik-111/professional/Data_Science/projects/CRSystem/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/karthik-111/professional/Data_Science/projects/CRSystem'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    model_name: str
    target_column: str

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            model_name = config.model_name,
            target_column = schema.name
            
        )

        return model_trainer_config


In [8]:
import pandas as pd
import os
from mlProject import logger
from sklearn.ensemble import RandomForestClassifier
import joblib

In [12]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]

        rand = RandomForestClassifier()
        rand.fit(train_x, train_y)

        joblib.dump(rand, os.path.join(self.config.root_dir, self.config.model_name))

        return rand



In [13]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
except Exception as e:
    raise e

[2024-03-25 10:46:18,072: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-25 10:46:18,078: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-25 10:46:18,081: INFO: common: created directory at: artifacts]
[2024-03-25 10:46:18,082: INFO: common: created directory at: artifacts/model_trainer]


/home/karthik-111/miniconda3/envs/crs/lib/python3.12/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [16]:
import joblib

In [17]:
ds_cols=joblib.load("artifacts/data_transformation/ds_cols.joblib")

In [19]:
predict_values = {}
for i in ds_cols:   
    predict_values[i]=None

predict_values["Online"] =   0 
predict_values["Total_Tuition_Cost"]= 60000
predict_values["Program_Years_Full_Time"]= 1.0
predict_values["Need_GRE"]= 0 
predict_values["Institution Type"]= 0

for i in predict_values.keys():
    if i == "City_"+"New York City":
        predict_values[i]=1
    index = range(1)

data=pd.DataFrame(predict_values,index=index)
data.fillna(0,inplace=True)   
data



/tmp/ipykernel_36291/4186083598.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna(0,inplace=True)


,Online,Total_Tuition_Cost,Program_Years_Full_Time,Need_GRE,Institution Type,City_Albany,City_Ann Arbor,City_Atlanta,City_Auburn,City_Austin,...,City_Stockton,City_Stony Brook,City_Storrs,City_Syracuse,City_Tallahassee,City_Tempe,City_Tucson,City_University Park,City_Washington,City_Worcester
0,0,60000,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
from sklearn.preprocessing import StandardScaler

In [29]:
sc = StandardScaler()
data[["Total_Tuition_Cost"]]=sc.fit_transform(data[["Total_Tuition_Cost"]])
data

,Online,Total_Tuition_Cost,Program_Years_Full_Time,Need_GRE,Institution Type,City_Albany,City_Ann Arbor,City_Atlanta,City_Auburn,City_Austin,...,City_Stockton,City_Stony Brook,City_Storrs,City_Syracuse,City_Tallahassee,City_Tempe,City_Tucson,City_University Park,City_Washington,City_Worcester
0,0,0.0,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
model_trainer_config.train().predict(data)[0]

/home/karthik-111/miniconda3/envs/crs/lib/python3.12/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/karthik-111/miniconda3/envs/crs/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


'Pace University'